# Tests file

In this file we will make performance and consistency tests.

In [40]:
#import sys
#!conda install --yes --prefix {sys.prefix} -c conda-forge gensim

import time
import pickle
import Globals.globals as glob
import SearchAlgorithms.searchAlgorithms as algo
from Tokenization import tokenizer
from Tokenization.tokenizer import createListOfTokens, replaceWordsByStem, replaceWordsByLemma, removeStopWords
from QueryMaker.queryShell import processQueryString
from DocumentServer import documentServer
from Tokenization.TokenizationCpp import tokenizer as tokenizerCpp
from IFConstruction import ifConstructor

datasetFoldername = "../latimes"
documentServer.foldername = datasetFoldername
glob.loadDocID2Content()

## Consistency tests

### 1. Impact of the word score on the top 10 documents

Setting : 
    - search algorithm : naive,
    - inverted file : no stemming and no lemmatization,
    - query processing : no stemming, no lemmatization, no word embedding.
    - query : "Chocolate and internet"
    
Variable parameter : **word score ∈ {<number of occurence\>, <tf * idf>}**
 
In this section, we compute the top 10 results with the naive algorithm using an inverted file which has been built without any stemming, lemmatization and no word embedding is applied on the query.
We think "Chocolate and internet" is a relevant query to test the word score since there is a significant difference between the number of occurence of "chocolate" and "internet" in the dataset as shown further.

In [3]:
searchAlgorithm = algo.naiveAlgo
query = "Chocolate and internet"
query = processQueryString(query, stemming = False, lemmatization = False, embedding = False)
print(query)

[('chocolate', 3), ('internet', 3)]


Note : because no word embedding is used, the reader must ignore the weights paired with the words. The weights are not used by the naive algorithm anyway.

* **First test : word score = number of occurence** 

In [4]:
vocabulary_filename = "Globals/nostemm_nolemm_notfidf/vocabulary.dict"
IF_filename = "Globals/nostemm_nolemm_notfidf/IF.dict"

glob.loadVocabulary(vocabulary_filename, IF_filename)

choco_PL = glob.voc2PostingList("chocolate")
internet_PL = glob.voc2PostingList("internet")

print("len(choco_PL) :", len(choco_PL))
print("len(internet_PL) :", len(internet_PL))

print("list(choco_PL.items())[:4] :", list(choco_PL.items())[:4])
print("list(internet_PL.items())[:4] :", list(internet_PL.items())[:4])

The vocabulary set has a size of 289292
len(choco_PL) : 723
len(internet_PL) : 4
list(choco_PL.items())[:4] : [('321713', 38), ('145821', 27), ('321712', 25), ('111', 24)]
list(internet_PL.items())[:4] : [('85032', 8), ('85141', 6), ('105932', 1), ('254071', 1)]


Note : we can observe that "chocolate" appears in more documents and with a bigger number of occurence in each documents.

In [4]:
result = searchAlgorithm(query)

content_result = documentServer.serveDocuments(result)

for idx, doc in enumerate(content_result.keys()):
	print(idx+1,"----------------------------------")
	print(content_result[doc]["metadata"]),
print("----------------------------------")

1 ----------------------------------
DOCID : 321713

DATE : December 13, 1990, Thursday, Home Edition 

SECTION : Food; Part H; Page 20; Column 1 

HEADLINE : GOOD COOKING: MAKE YOUR HOLIDAY INDULGENCE BITTERSWEET 

2 ----------------------------------
DOCID : 145821

DATE : December 8, 1989, Friday, Orange County Edition 

SECTION : Orange County Life; Part N; Page 11; Column 1 

HEADLINE : SHE FINDS SWEET SUCCESS WITH CHOCOLATES 

3 ----------------------------------
DOCID : 321712

DATE : December 13, 1990, Thursday, Home Edition 

SECTION : Food; Part H; Page 20; Column 1 

HEADLINE : BACK TO BASICS: DON'T BE AFRAID: IT'S SIMPLY PERFECT CHOCOLATE 

4 ----------------------------------
DOCID : 111

DATE : January 1, 1989, Sunday, Home Edition 

SECTION : Opinion; Part 5; Page 5; Column 1; Op-Ed Desk 

HEADLINE : LITTLE CHOCOLATE DOUGHNUTS TELL THE TALE OF THE U.S. TRADE CRISIS 

5 ----------------------------------
DOCID : 196334

DATE : March 29, 1990, Thursday, Home Edition 

SECT

Note : if we look at the headlines of the top 10 results we can clearly see that they all are related to "chocolate".

* **Second test : word score = tf * idf = (1 + log(number of occurrences)) * log(total number of documents/(1 + length of posting list)))** 

In [25]:
vocabulary_filename = "Globals/nostemm_nolemm_tf_idf/vocabulary.dict"
IF_filename = "Globals/nostemm_nolemm_tf_idf/IF.dict"

glob.loadVocabulary(vocabulary_filename, IF_filename)

choco_PL = glob.voc2PostingList("chocolate")
internet_PL = glob.voc2PostingList("internet")

print("len(choco_PL) :", len(choco_PL))
print("len(internet_PL) :", len(internet_PL))

print("list(choco_PL.items())[:4] :", list(choco_PL.items())[:4])
print("list(internet_PL.items())[:4] :", list(internet_PL.items())[:4])

The vocabulary set has a size of 290045
len(choco_PL) : 724
len(internet_PL) : 4
list(choco_PL.items())[:4] : [('321713', 24.139), ('145821', 22.36), ('321712', 21.959), ('111', 21.747)]
list(internet_PL.items())[:4] : [('85032', 32.037), ('85141', 29.044), ('105932', 10.403), ('254071', 10.403)]


Note : we can observe that, even if "chocolate" appears in more documents and with a bigger number of occurence in each documents (as seen before), the new score computation makes "internet" reach higher scores than "chocolate" in some documents.

In [14]:
result = searchAlgorithm(query)

content_result = documentServer.serveDocuments(result)

for idx, doc in enumerate(content_result.keys()):
	print(idx+1,"----------------------------------")
	print(content_result[doc]["metadata"]),
print("----------------------------------")

1 ----------------------------------
DOCID : 85032

DATE : July 21, 1989, Friday, Home Edition 

SECTION : Part 1; Page 14; Column 5; National Desk 

HEADLINE : COMPUTER NETWORK SEEN AS STILL VULNERABLE TO VIRUSES 

2 ----------------------------------
DOCID : 85141

DATE : July 21, 1989, Friday, Orange County Edition 

SECTION : Business; Part 4; Page 3; Column 5; Financial Desk 

HEADLINE : PLAN SOUGHT TO KEEP 'VIRUSES' FROM A COMPUTER NETWORK 

3 ----------------------------------
DOCID : 321713

DATE : December 13, 1990, Thursday, Home Edition 

SECTION : Food; Part H; Page 20; Column 1 

HEADLINE : GOOD COOKING: MAKE YOUR HOLIDAY INDULGENCE BITTERSWEET 

4 ----------------------------------
DOCID : 145821

DATE : December 8, 1989, Friday, Orange County Edition 

SECTION : Orange County Life; Part N; Page 11; Column 1 

HEADLINE : SHE FINDS SWEET SUCCESS WITH CHOCOLATES 

5 ----------------------------------
DOCID : 321712

DATE : December 13, 1990, Thursday, Home Edition 

SECTION

Note : now, if we look at the headlines of the top 10 results, both "chocolate" and "internet" seem to be represented in the results. However, it is not obvious why documents related to "internet" should be better than the ones related to "chocolate". This behavior is due to the naive algorithm.

**The score tf * idf shows itself more relevant than a simple word occurence counter since it allows rarer words to be considered by the algorithm and it lower the importance of common words. From now on, our tests will only use this score.**

### 2. Impact of the search algorithm on the top 10 documents

Setting : 
    - word score : tf * idf,
    - inverted file : no stemming and no lemmatization,
    - query processing : no stemming, no lemmatization, no word embedding.
    - query : "Chocolate and internet"

Variable parameter : **search algorithm ∈ {<naive\>, <fagin\>, <treshold\>}**

In this section, we compute the top 10 results with two new search algorithms using an inverted file which has been built without any stemming, lemmatization and no word embedding is applied on the query.
We still use "Chocolate and internet" as the query.

Results from the naive algorithm have already been computed above.

In [15]:
for idx, doc in enumerate(content_result.keys()):
	print(idx+1,"----------------------------------")
	print(content_result[doc]["metadata"]),
print("----------------------------------")

1 ----------------------------------
DOCID : 85032

DATE : July 21, 1989, Friday, Home Edition 

SECTION : Part 1; Page 14; Column 5; National Desk 

HEADLINE : COMPUTER NETWORK SEEN AS STILL VULNERABLE TO VIRUSES 

2 ----------------------------------
DOCID : 85141

DATE : July 21, 1989, Friday, Orange County Edition 

SECTION : Business; Part 4; Page 3; Column 5; Financial Desk 

HEADLINE : PLAN SOUGHT TO KEEP 'VIRUSES' FROM A COMPUTER NETWORK 

3 ----------------------------------
DOCID : 321713

DATE : December 13, 1990, Thursday, Home Edition 

SECTION : Food; Part H; Page 20; Column 1 

HEADLINE : GOOD COOKING: MAKE YOUR HOLIDAY INDULGENCE BITTERSWEET 

4 ----------------------------------
DOCID : 145821

DATE : December 8, 1989, Friday, Orange County Edition 

SECTION : Orange County Life; Part N; Page 11; Column 1 

HEADLINE : SHE FINDS SWEET SUCCESS WITH CHOCOLATES 

5 ----------------------------------
DOCID : 321712

DATE : December 13, 1990, Thursday, Home Edition 

SECTION

Note : the top 10 (doc ID) with the naive algorithm is 85032, 85141, 321713, 145821, 321712, 111, 196334, 236382, 265773, 179442.

We compute the same query with the fagin algorithm.

In [23]:
searchAlgorithm = algo.faginAlgo

print(query)

result = searchAlgorithm(query)

content_result = documentServer.serveDocuments(result)

for idx, doc in enumerate(content_result.keys()):
	print(idx+1,"----------------------------------")
	print(content_result[doc]["metadata"]),
print("----------------------------------")

[('chocolate', 3), ('internet', 3)]
1 ----------------------------------
DOCID : 85032

DATE : July 21, 1989, Friday, Home Edition 

SECTION : Part 1; Page 14; Column 5; National Desk 

HEADLINE : COMPUTER NETWORK SEEN AS STILL VULNERABLE TO VIRUSES 

2 ----------------------------------
DOCID : 85141

DATE : July 21, 1989, Friday, Orange County Edition 

SECTION : Business; Part 4; Page 3; Column 5; Financial Desk 

HEADLINE : PLAN SOUGHT TO KEEP 'VIRUSES' FROM A COMPUTER NETWORK 

3 ----------------------------------
DOCID : 321713

DATE : December 13, 1990, Thursday, Home Edition 

SECTION : Food; Part H; Page 20; Column 1 

HEADLINE : GOOD COOKING: MAKE YOUR HOLIDAY INDULGENCE BITTERSWEET 

4 ----------------------------------
DOCID : 145821

DATE : December 8, 1989, Friday, Orange County Edition 

SECTION : Orange County Life; Part N; Page 11; Column 1 

HEADLINE : SHE FINDS SWEET SUCCESS WITH CHOCOLATES 

5 ----------------------------------
DOCID : 321712

DATE : December 13, 19

Note : the top 10 (doc ID) with the fagin algorithm is 85032, 85141, 321713, 145821, 321712, 111, 196334, 236382, 265773, 179442. It is exactly the same than with the naive algorithm.

We compute the same query with the threshold algorithm.

In [26]:
searchAlgorithm = algo.threshold

result = searchAlgorithm(query)

content_result = documentServer.serveDocuments(result)

for idx, doc in enumerate(content_result.keys()):
	print(idx+1,"----------------------------------")
	print(content_result[doc]["metadata"]),
print("----------------------------------")

1 ----------------------------------
DOCID : 85032

DATE : July 21, 1989, Friday, Home Edition 

SECTION : Part 1; Page 14; Column 5; National Desk 

HEADLINE : COMPUTER NETWORK SEEN AS STILL VULNERABLE TO VIRUSES 

2 ----------------------------------
DOCID : 85141

DATE : July 21, 1989, Friday, Orange County Edition 

SECTION : Business; Part 4; Page 3; Column 5; Financial Desk 

HEADLINE : PLAN SOUGHT TO KEEP 'VIRUSES' FROM A COMPUTER NETWORK 

3 ----------------------------------
DOCID : 321713

DATE : December 13, 1990, Thursday, Home Edition 

SECTION : Food; Part H; Page 20; Column 1 

HEADLINE : GOOD COOKING: MAKE YOUR HOLIDAY INDULGENCE BITTERSWEET 

4 ----------------------------------
DOCID : 145821

DATE : December 8, 1989, Friday, Orange County Edition 

SECTION : Orange County Life; Part N; Page 11; Column 1 

HEADLINE : SHE FINDS SWEET SUCCESS WITH CHOCOLATES 

5 ----------------------------------
DOCID : 321712

DATE : December 13, 1990, Thursday, Home Edition 

SECTION

Note : the top 10 (doc ID) with the threshold algorithm is 85032, 85141, 321713, 145821, 321712, 111, 196334, 236382, 265773, 324683. It is almost the same than with the two previous algorithms except for the last result 324683 here. This last result is still relevant for the query though. 

**When we look at the consistency of the results computed with the three algorithms they all are similar. The threshold algorithm adds incertainty in the results since it doesn't loop through the whole inverted file and thus may provides better performance (see performance tests below). But because the concern of this first tests is consistency we will now use the fagin algorithm.**

### Impact of stemming, lemmatization and word embedding

Setting : 
    - word score : tf * idf,
    - search algorithm : fagin,
    - query : "Chocolate and feet"

Variable parameter : **word processing ∈ {<stemming\>, <lemmatization\>, <word embedding\>}**

In this section, we compute the top 10 results with the fagin algorithms using an inverted file and a query, both being  built with different word processing techniques.
The query for this test is "Chocolate and feet", "feet" allows us to show the difference between stemming and lemmatization because of its root.

If we don't use stemming, lemmatization or word embedding we obtain the following results.

In [27]:
def applyFaginOnQuery(processedQuery):
    queryResult = algo.faginAlgo(processedQuery)
    if(queryResult):
        returnedDocuments = documentServer.serveDocuments(queryResult)
        print("\n")
        print("results:\n")
        for idx, doc in enumerate(returnedDocuments.keys()):
            print(idx+1,"----------------------------------")
            print(returnedDocuments[doc]["metadata"]),
            print("----------------------------------")
    else:
        print("no result\n")

In [29]:
vocabulary_filename = "./Globals/nostemm_nolemm_tf_idf/vocabulary.dict"
IF_filename = "./Globals/nostemm_nolemm_tf_idf/IF.dict"
glob.loadVocabulary(vocabulary_filename,IF_filename)

print("The vocabulary set has a size of", len(glob.vocabularyDict))

query = "Chocolate and feet"

# Apply stemming on the query
processedQuery = processQueryString(query)
print(processedQuery)

# Apply fagin algorithm
applyFaginOnQuery(processedQuery)

The vocabulary set has a size of 290045
290045
[('chocolate', 3), ('feet', 3)]


results:

1 ----------------------------------
DOCID : 110992

DATE : September 22, 1989, Friday, Orange County Edition 

SECTION : Calendar; Part 6; Page 23; Column 1; Entertainment Desk 

HEADLINE : RESTAURANTS / MAX JACOBSON; 
</P>
<P>
CHINESE DINING EXPERIENCE IS EASY ON THE EYE, LESS SO ON PALATE, POCKETBOOK 

----------------------------------
2 ----------------------------------
DOCID : 207671

DATE : April 22, 1990, Sunday, Home Edition 

SECTION : Magazine; Page 21; Magazine Desk 

HEADLINE : THE PLAYGROUND BECOMES THE BATTLEGROUND 

----------------------------------
3 ----------------------------------
DOCID : 323491

DATE : December 17, 1990, Monday, Valley Edition 

SECTION : Metro; Part B; Page 3; Column 1 

HEADLINE : 2 VOICES CLAIM THEY SPEAK FOR ENCINO; 
</P>
<P>
RIVALRY: THE 10-SQUARE-MILE AREA'S PROPERTY OWNERS CAN CHOOSE FROM GROUPS WHOSE 
APPROACHES TO DEVELOPMENT AND OTHER ISSUES CLAS

Result obtained:

    The vocabulary set has a size of  290045

    [('chocolate', 3), ('feet', 3)]
    
    Top 10 : 110992, 207671, 323491, 74671, 247462, 30071, 53702, 5461, 134434, 214793


We will now add stemming processing on the inverted file and on the user query.

In [31]:
vocabulary_filename = "./Globals/stemm_nolemm_tfidf/vocabulary.dict"
IF_filename = "./Globals/stemm_nolemm_tfidf/IF.dict"
glob.loadVocabulary(vocabulary_filename,IF_filename)

print("The vocabulary set has a size of", len(glob.vocabularyDict))

query = "Chocolate and feet"

# Apply stemming on the query
processedQuery = processQueryString(query,stemming = True)
print(processedQuery)

# Apply fagin algorithm
applyFaginOnQuery(processedQuery)

The vocabulary set has a size of 234118
The vocabulary set has a size of 234118
[('chocol', 3), ('feet', 3)]


results:

1 ----------------------------------
DOCID : 110992

DATE : September 22, 1989, Friday, Orange County Edition 

SECTION : Calendar; Part 6; Page 23; Column 1; Entertainment Desk 

HEADLINE : RESTAURANTS / MAX JACOBSON; 
</P>
<P>
CHINESE DINING EXPERIENCE IS EASY ON THE EYE, LESS SO ON PALATE, POCKETBOOK 

----------------------------------
2 ----------------------------------
DOCID : 207671

DATE : April 22, 1990, Sunday, Home Edition 

SECTION : Magazine; Page 21; Magazine Desk 

HEADLINE : THE PLAYGROUND BECOMES THE BATTLEGROUND 

----------------------------------
3 ----------------------------------
DOCID : 323491

DATE : December 17, 1990, Monday, Valley Edition 

SECTION : Metro; Part B; Page 3; Column 1 

HEADLINE : 2 VOICES CLAIM THEY SPEAK FOR ENCINO; 
</P>
<P>
RIVALRY: THE 10-SQUARE-MILE AREA'S PROPERTY OWNERS CAN CHOOSE FROM GROUPS WHOSE 
APPROACHES TO DEV

Result obtained:

    The vocabulary set has a size of  234118

    [('chocol', 3), ('feet', 3)]
    
    Top 10 : 110992, 207671, 323491, 74671, 247462,30071, 53702, 5461, 103552, 134434


We will now add the lemmatization procedure to tokens in the inverted file and in the query.

In [32]:
vocabulary_filename = "./Globals/stemm_lemm_tfidf/vocabulary.dict"
IF_filename = "./Globals/stemm_lemm_tfidf/IF.dict"
glob.loadVocabulary(vocabulary_filename,IF_filename)

print("The vocabulary set has a size of", len(glob.vocabularyDict))

query = "Chocolate and feet"

# Apply stemming on the query
processedQuery = processQueryString(query,lemmatization = True)
print(processedQuery)

# Apply fagin algorithm
applyFaginOnQuery(processedQuery)

The vocabulary set has a size of 230710
The vocabulary set has a size of 230710
[('chocol', 3), ('foot', 3)]


results:

1 ----------------------------------
DOCID : 207562

DATE : April 21, 1990, Saturday, Orange County Edition 

SECTION : Orange County Life; Part N; Page 2; Column 1 

HEADLINE : HOW MUCH 'I DO' 

----------------------------------
2 ----------------------------------
DOCID : 207671

DATE : April 22, 1990, Sunday, Home Edition 

SECTION : Magazine; Page 21; Magazine Desk 

HEADLINE : THE PLAYGROUND BECOMES THE BATTLEGROUND 

----------------------------------
3 ----------------------------------
DOCID : 110992

DATE : September 22, 1989, Friday, Orange County Edition 

SECTION : Calendar; Part 6; Page 23; Column 1; Entertainment Desk 

HEADLINE : RESTAURANTS / MAX JACOBSON; 
</P>
<P>
CHINESE DINING EXPERIENCE IS EASY ON THE EYE, LESS SO ON PALATE, POCKETBOOK 

----------------------------------
4 ----------------------------------
DOCID : 247462

DATE : July 15, 1990,

Result obtained:

    The vocabulary set has a size of  230710

    [('chocol', 3), ('foot', 3)]
    
    Top 10 : 207562, 207671, 110992, 247462, 103552, 323491, 134434, 295563, 74671, 30071


Finally we will extend the query with 3 synonyms for each tokens using word embedding.

In [33]:
vocabulary_filename = "./Globals/stemm_lemm_tfidf/vocabulary.dict"
IF_filename = "./Globals/stemm_lemm_tfidf/IF.dict"
glob.loadVocabulary(vocabulary_filename,IF_filename)

embeddingFile = open('./Globals/embeddingModel', 'rb')
model = pickle.load(embeddingFile)
embeddingFile.close()

query = "Chocolate and feet"

# Apply stemming on the query
processedQuery = processQueryString(query,lemmatization = True, embedding = True, embeddingModel = model, nbOfSynonyms = 3)
print(processedQuery)

# Apply fagin algorithm
applyFaginOnQuery(processedQuery)

The vocabulary set has a size of 230710
[('chocol', 3), ('foot', 3), ('caramel', 1), ('eclair', 1), ('cake', 1), ('inch', 1), ('mile', 1), ('diamet', 1)]


results:

1 ----------------------------------
DOCID : 229922

DATE : June 7, 1990, Thursday, Home Edition 

SECTION : Food; Part H; Page 1; Column 3 

HEADLINE : WINE COUNTRY CHEFS; 
</P>
<P>
THREE YEARS AGO ONLY SIX WINERIES IN NAPA AND SONOMA HAD RESIDENT CHEFS. TODAY 
THE NUMBER HAS TRIPLED. THESE ARE RISING STARS. 

----------------------------------
2 ----------------------------------
DOCID : 141533

DATE : November 30, 1989, Thursday, Home Edition 

SECTION : Food; Part H; Page 2; Column 1 

HEADLINE : EASY-TO-MAKE GIFTS THAT ARE STRAIGHT FROM THE HEART AND THE KITCHEN; 
</P>
<P>
HOLIDAYS: FOODS THAT FREEZE WELL OR DON'T REQUIRE REFRIGERATION MAKE THE SAFEST 
EDIBLE PRESENTS. 

----------------------------------
3 ----------------------------------
DOCID : 179442

DATE : February 22, 1990, Thursday, Home Edition 

SECTION : 

Result obtained:

    [('chocol', 3), ('foot', 3), ('caramel', 1), ('eclair', 1), ('cake', 1), ('inch', 1), ('mile', 1), ('diamet', 1)]
    
    Top 10 : 22922, 141533, 179442, 321713, 132143, 209494, 297811, 148224, 271941, 321712


**Steming and lemmatization permits to decrease the vocabulary size, especially stemming.**

**We can observe that synonyms given by the word embedding model are relevant.**

**Stemming doesn't change the results from the fagin algorithms whereas lemmatization changes some documents retrieved.
Word embedding permits to extend queries and retrieve new documents.**

## Performance tests

### 1. Time to build and query the inverted file

In this section, we don't use any word processing techniques (stemming, lemmatization, word embedding).

Firstly we will build the inverted file over the whole data set in RAM memory then on disk.

In [41]:
import cProfile

In [ ]:
tokenizer_ = tokenizer.Tokenizer(datasetFoldername)

#cProfile.run("ifConstructor.constructIF(tokenizer_, stemming = False, lemmatization = False, wordEmbedding = False)")

![alt text](images/inramprofile.png)

Note : cProfile gives us a cpu time of 567 seconds which is roughly equivalent to 10 minutes.

![alt text](images/inram13gb.png)

Note : once the construction is done we can see that the hashmap takes around 1300 MB in RAM.

In [ ]:
tokenizer_ = tokenizerCpp.Tokenizer(datasetFoldername, lemmatization_ = False, stemming_ = False)
#set runSize such that :
#the total number of documents (~130 000) in the dataset divided by runSize is less than the allowed number of 
#simultaneously opened files on your machine (usually 1024) 
runSize_ = 10000

#cProfile.run("ifConstructor.constructIF_diskBased(tokenizer_, runSize = runSize_, score_tf_idf = True)")

![alt text](images/run10000profile.png)

Note : cProfile gives us a cpu time of 1123 seconds which is roughly equivalent to 19 minutes.

![alt text](images/run10000.png "Title")

Note : from the system monitor we can see that the program reach 583 MB of RAM usage. The upbound is reached during the creation of the temporary files, before the merge operation. 

In [ ]:
runSize_ = 130

cProfile.run("ifConstructor.constructIF_diskBased(tokenizer_, runSize = runSize_, score_tf_idf = True)")

![alt text](images/run130profile.png)

Note : cProfile gives us a cpu time of 5657 seconds which is roughly equivalent to 1h34.

![alt text](images/run130mergemonitor.png)

Note : from the system monitor we can see that the program reach 137 MB of RAM usage. In this case the upbound is reached during the merge operation.

**The bigger the size of a run the more data will be charged in memory before a flush operation. But, as shown above, the computation time is improved by more RAM usage.**


## Time to run algorithm
In this section, we will use neither stemming/lemmatization nor word embedding. 

We compute the three algos with same queries of different types:
the queries can be not existing in our dictionary, very short, or very long.



### function to record the test time:

-----------------------------

In [4]:
def testTime(queries):
    start_time = time.time()
    for query in queries:
        algo.naiveAlgo(query)
    print("--- %s naiveAlgo seconds ---" % (time.time() - start_time))
    start_time = time.time()
    for query in queries:
        algo.faginAlgo(query)
    print("--- %s faginAlgo seconds ---" % (time.time() - start_time))
    start_time = time.time()
    for query in queries:
        algo.threshold(query)
    print("--- %s threshold seconds ---" % (time.time() - start_time))


In [49]:
glob.loadVocabulary("./Globals/nostemm_nolemm_tf_idf/vocabulary.dict","./Globals/nostemm_nolemm_tf_idf/IF.dict")

### The queries: (the powers are all set to 3 to avoid the effect of word embedding)

In [2]:
oneWord = [
        [("daylight",3)]
        ]

notExist = [[("fdadfdfewf",3)],
           [("114rf4434",3)],
            [("jdifjoiq2323",3)]
           ]

queries = [
                [("love",3), ("chocolate",3)],
                [("january",3)],
                [("narrow",3)],
                [("today",3), ("tomorrow",3)]           
        ]

queries1 = [
                [("love",3), ("and",3), ("chocolate",3)],
                [("january",3)],
                [("narrow",3)],
                [("today",3), ("and",3), ("tomorrow",3)],
                [("good",3)],
                [("better",3)],
                [("decent",3),("life",3), ("time",3), ("evening",3)]
            ]


We compute the three algos on the words not existing in the dict:

In [57]:
testTime(notExist)

--- 4.124641418457031e-05 naiveAlgo seconds ---
--- 0.0005621910095214844 faginAlgo seconds ---
--- 0.00039196014404296875 threshold seconds ---


Result obtained:

--- 4.124641418457031e-05 naiveAlgo seconds ---  
--- 0.0005621910095214844 faginAlgo seconds ---  
--- 0.00039196014404296875 threshold seconds ---

We compute the three algos with one word 

In [5]:
testTime(oneWord)

NameError: name 'time' is not defined

Result obtained: 
  
--- 0.002989053726196289 naiveAlgo seconds ---  
--- 0.003451824188232422 faginAlgo seconds ---  
--- 0.001968860626220703 threshold seconds ---

We compute the three algos with random words
Remark: We notice that the fagin algo is quite slow because it needs to go through every posting list

In [59]:
testTime(queries)

--- 0.45010828971862793 naiveAlgo seconds ---
--- 8.133760929107666 faginAlgo seconds ---
--- 0.44022607803344727 threshold seconds ---


Result obtained:


--- 0.45010828971862793 naiveAlgo seconds ---  
--- 8.133760929107666 faginAlgo seconds ---  
--- 0.44022607803344727 threshold seconds ---

In [ ]:
testTime(queries1)

Result obtained:

--- 1.6027929782867432 naiveAlgo seconds ---  
--- 92.30611991882324 faginAlgo seconds ---  
--- 9.01853895187378 threshold seconds ---

### Conclusion:
    

#### Why naive algo is so rapid at certain cases?

we see a very efficient naive algo because in python the sorted function is implemented with binary tree data structure which reduces the time complexity to O(nlogn). The cost of this acceleration is the memory consumed during the calculation.



#### Why Fagin algo is so slow in certain cases?

Because the fagin algo will continue running if it doesn't find an object which is been listed in all posting lists until it runs through one posting list. In our case, it doesn't exist one doc which has all the terms and the posting list can be very long. That's why the fagin algo can be very slow.


#### Comparison of TA and FA:

-TA sees less objects than FA and it stops at least as early as FA

-TA requires only bounded buffer space while FA makes use of unbouded buffers

#### Best Algo?

It depends on the aggregation functions characteristics and our database restrictions.
In our case, the aggregation function is strictly monotone so we can we can say that TA is the best algo in general cases.  

But if the aggregation function changes, or the volume of the database changes, there will be more chances that the FA or TA won't give us a correct results.
  
  
  


  
  
  
Remark: the performance test is realized on a machine of 16 GB memory(2400 MHz) of 2.6 GHz 6-Core Intel Core i7